In [5]:
import pandas as pd
import time
from web3 import Web3
import numpy as np # Import numpy as it's used for data processing

# Connect to local Ganache blockchain
ganache_url = "http://127.0.0.1:7545"  # Update to 8545 if needed
web3 = Web3(Web3.HTTPProvider(ganache_url))

# Verify connection
if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
else:
    print("❌ Connection failed. Ensure Ganache is running.")
    exit()

# Load the smart contract
# IMPORTANT: Replace with your CURRENTLY DEPLOYED contract address from Remix.
# Example: contract_address = "0xd8b934580fcE35a11B58C6D73aDeE468a2833fa8"
# The address 0xF7bA4959568973D4c9CB36678D809D277F6D1276 was previously used.
# Please ensure you use the one currently deployed in your Ganache instance.
contract_address = "0xF7bA4959568973D4c9CB36678D809D277F6D1276" # <<< VERIFY THIS ADDRESS
# Ensure all 'false' and 'true' are 'False' and 'True' in your ABI
abi = [
  {
		"inputs": [],
		"stateMutability": "nonpayable",
		"type": "constructor"
	},
	{
		"anonymous": False,
		"inputs": [
			{
				"indexed": False,
				"internalType": "uint256",
				"name": "timestamp",
				"type": "uint256"
			},
			{
				"indexed": False,
				"internalType": "string",
				"name": "packageId",
				"type": "string"
			},
			{
				"indexed": False,
				"internalType": "string",
				"name": "dataType",
				"type": "string"
			},
			{
				"indexed": False,
				"internalType": "string",
				"name": "dataValue",
				"type": "string"
			}
		],
		"name": "DataStored",
		"type": "event"
	},
	{
		"inputs": [],
		"name": "MAX_ENTRIES",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "dataRecords",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "timestamp",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "packageId",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "dataType",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "dataValue",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "getRecord",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "getTotalRecords",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "owner",
		"outputs": [
			{
				"internalType": "address",
				"name": "",
				"type": "address"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "string",
				"name": "_packageId",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_dataType",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_dataValue",
				"type": "string"
			}
		],
		"name": "storeData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	}
]
contract = web3.eth.contract(address=contract_address, abi=abi)
web3.eth.default_account = web3.eth.accounts[0] # Set default sender (first Ganache account, must be owner)
print(f"✅ Connected to Smart Contract at {contract_address}")

# --- Test contract by calling getTotalRecords (initial check) ---
try:
    total_records_initial = contract.functions.getTotalRecords().call()
    print(f"\nTotal Records on blockchain (initial check): {total_records_initial}")
except Exception as e:
    print(f"❌ Error calling getTotalRecords: {e}")
    print("Please ensure contract is deployed correctly and chain synced. Exiting.")
    exit()

# --- Store dummy IoT data if no records exist (for testing purposes) ---
if total_records_initial == 0:
    print("\nNo records found. Storing some dummy data to populate the contract...")
    try:
        txn = contract.functions.storeData("INITIAL001", "Temp", "25.0C").transact({
            'from': web3.eth.default_account,
            'gas': 1000000
        })
        receipt = web3.eth.wait_for_transaction_receipt(txn)
        if receipt.status == 1:
            # Decode event for the timestamp
            event_logs = contract.events.DataStored().process_receipt(receipt)
            if event_logs:
                dummy_timestamp = event_logs[0].args.timestamp
                readable_dummy_timestamp = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(dummy_timestamp))
                print(f"✅ Dummy data stored on blockchain! Timestamp: {readable_dummy_timestamp}")
            else:
                print("✅ Dummy data stored on blockchain! (Timestamp not found in event logs)")
        else:
            print("❌ Dummy data transaction failed!")
            print(receipt)
            exit()
    except Exception as e:
        print(f"❌ Error storing initial dummy data: {e}")
        print("Please ensure your default account is the contract owner.")
        exit()
    # Update total_records after storing dummy data
    total_records_initial = contract.functions.getTotalRecords().call()
    print(f"Total Records on blockchain (after initial dummy data): {total_records_initial}")

# --- Load IoT sensor data from CSV and populate blockchain (if desired, or use existing data) ---
# This section assumes you want to add more data from a CSV.
# If your contract is already populated and you just want to retrieve, you can comment this out.
try:
    df_upload = pd.read_csv("formatted_logistics_data.csv")
    print("\nCSV Data Preview for Upload:")
    print(df_upload.head())

    # Function to send IoT data to blockchain
    def send_iot_data(package_id, data_type, data_value):
        """Sends IoT data to the deployed smart contract and prints its timestamp."""
        try:
            txn = contract.functions.storeData(package_id, data_type, data_value).transact({
                'from': web3.eth.default_account,
                'gas': 3000000
            })
            receipt = web3.eth.wait_for_transaction_receipt(txn)

            if receipt.status == 1:
                event_logs = contract.events.DataStored().process_receipt(receipt)
                if event_logs:
                    timestamp = event_logs[0].args.timestamp
                    readable_timestamp = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timestamp))
                    print(f"✅ Data Stored: {data_type} - {data_value}, Timestamp: {readable_timestamp}, Txn Hash: {receipt.transactionHash.hex()}")
                else:
                    print(f"✅ Data Stored: {data_type} - {data_value}, Txn Hash: {receipt.transactionHash.hex()} (Timestamp not found in event logs)")
            else:
                print(f"❌ Data Stored: {data_type} - {data_value}, Transaction failed! Txn Hash: {receipt.transactionHash.hex()}")
                print(receipt)
        except Exception as e:
            print(f"❌ Error storing data for {package_id}: {e}")

    print("\n--- Sending CSV data to Blockchain ---")
    for _, row in df_upload.iterrows():
        send_iot_data(str(row["device_id"]), str(row["data_type"]), str(row["data_value"]))
        time.sleep(0.5)  # Shorter delay to prevent flooding transactions
    print("--- Finished sending CSV data ---")

except FileNotFoundError:
    print("\n'formatted_logistics_data.csv' not found. Skipping CSV data upload.")
except Exception as e:
    print(f"❌ Error processing CSV for upload: {e}")


# --- Week 6: Data Retrieval and Processing ---
print("\n--- Starting Week 6: Data Retrieval and Processing ---")

# Get the total number of stored records (re-fetch after any uploads)
try:
    total_records = contract.functions.getTotalRecords().call()
    print(f"Total IoT records stored for retrieval: {total_records}")
except Exception as e:
    print(f"❌ Error getting total records for retrieval: {e}")
    exit()

if total_records == 0:
    print("No records to retrieve from the blockchain. Exiting processing.")
    exit()

# Fetch all stored IoT data and structure it in a DataFrame
data_list = []
for i in range(total_records):
    try:
        # Call getRecord to retrieve the data
        record = contract.functions.getRecord(i).call()
        # record will be a tuple: (timestamp, packageId, dataType, dataValue)
        data_list.append({
            "timestamp": record[0],
            "device_id": record[1], # Column name for DataFrame, corresponds to contract's packageId
            "data_type": record[2],
            "data_value": record[3]
        })
    except Exception as e:
        print(f"❌ Error retrieving record at index {i}: {e}")
        print("Skipping this record.")
        continue # Continue to the next record even if one fails

# Convert to a DataFrame
df = pd.DataFrame(data_list)

# Convert timestamp (Unix seconds) to readable datetime format
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")

# Display first few records of the raw DataFrame
print("\nDataFrame from Blockchain (first 5 records):")
print(df.head())

# --- Data Preprocessing ---
print("\n--- Preprocessing Retrieved Data ---")

# Extract numeric values from 'data_value' where applicable
# Robust regex for numbers, including decimals and scientific notation.
df["numeric_value"] = pd.to_numeric(
    df["data_value"].astype(str).str.extract(r'([+\-]?\d*\.?\d+(?:[Ee][+\-]?\d+)?)')[0],
    errors='coerce' # Convert non-numeric values to NaN
)

# Identify missing values after numeric extraction
print(f"Missing values before handling:\n{df.isnull().sum()}")

# Handle missing values in 'numeric_value' column
# FIX APPLIED HERE: Avoid chained assignment with inplace=True
# Replace NaNs in 'numeric_value' with 0 as per instructions for minor missing values.
df["numeric_value"] = df["numeric_value"].fillna(0) # Corrected line

# Display cleaned data with new 'numeric_value' column
print("\nDataFrame after Preprocessing (first 5 records):")
print(df.head())


# --- Verify all stored data (optional, but good for inspection) ---
print(f"\nTotal IoT records in DataFrame: {len(df)}")
if len(df) > 0:
    print("\n--- All Stored Records in DataFrame ---")
    # You can print the whole DataFrame if it's not too large
    # print(df.to_string()) # Use .to_string() for full view without truncation
    # Or just print a summary
    print(df)
    print("-" * 30)
else:
    print("No records to display after processing.")


# --- Save to CSV ---
output_csv_filename = "cleaned_iot_data.csv"
df.to_csv(output_csv_filename, index=False)

print(f"\n✅ Cleaned IoT data saved successfully as {output_csv_filename}")

print("\n--- Next Steps for GitHub Upload ---")
print("1. Save your current Python script (.py) or Jupyter Notebook (.ipynb).")
print(f"2. Locate the generated CSV file: {output_csv_filename} in your project directory.")
print("3. Navigate to your GitHub repository in your web browser: https://github.com/agalilo22/FIMS")
print("4. Click 'Add file' -> 'Upload files'.")
print("5. Drag and drop or select your Python script/Jupyter Notebook and the 'cleaned_iot_data.csv' file.")
print("6. Add a concise commit message (e.g., 'Feat: Week 6 Data Retrieval & Processing').")
print("7. Click 'Commit changes' to save your work to the repository.")

✅ Connected to Ganache successfully!
✅ Connected to Smart Contract at 0xF7bA4959568973D4c9CB36678D809D277F6D1276

Total Records on blockchain (initial check): 65

CSV Data Preview for Upload:
  device_id data_type    data_value
0    PKG001  Location      New York
1    PKG001    Status    In Transit
2    PKG002  Location       Chicago
3    PKG002    Status  At Warehouse
4    PKG003    Status     Delivered

--- Sending CSV data to Blockchain ---
✅ Data Stored: Location - New York, Timestamp: 2025-06-19 16:08:38, Txn Hash: 49279913a0bd1cb32ac796109e65a10c918401156754d9238a0573b226f714b7
✅ Data Stored: Status - In Transit, Timestamp: 2025-06-19 16:08:38, Txn Hash: 79304e2cf5e4258b0d346fece1f9c7d58c081604da0798718ba2ebc95eb732fd
✅ Data Stored: Location - Chicago, Timestamp: 2025-06-19 16:08:39, Txn Hash: 6e896d267f040c975b23f54c6a525948fec1a4ecfa7626c1d7168384b6606b63
✅ Data Stored: Status - At Warehouse, Timestamp: 2025-06-19 16:08:39, Txn Hash: 17d292768b4e94558658b1d90c730a9c4c89ad7c5459